<a href="https://colab.research.google.com/github/aayush2710/CTW2019-IDP/blob/master/Polynomial_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%tensorflow_version 1.x
import numpy as np # linear algebra
import tensorflow as tf
import numpy as np
import os
import h5py
import pandas as pd
import time
import matplotlib.pyplot as plt


In [ ]:
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, LeakyReLU, Input, Conv2D, AveragePooling2D, MaxPooling2D, Dropout, Flatten, concatenate
%load_ext tensorboard

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import h5py

def load_data(path):
    f = h5py.File(path, 'r')
    obj = iter(f.keys())
    #H = f['dataset_1'][:]
    H = f[next(obj)][:]
    f.close()
    return H

def load_features(path):
    f = h5py.File(path, 'r')
    print(f.keys())
    H = f['dataset_1'][:]
    f.close()
    return H


In [ ]:
Y_pos = load_data('/content/drive/My Drive/CTW_2019/r_Position_CTW_Train.h5')
SNR = load_data('/content/drive/My Drive/CTW_2019/SNR_CTW_Train.h5')
h_COMP = load_features('/content/drive/My Drive/CTW_2019/data2019.h5')


In [ ]:
print(h_COMP.shape)
print(SNR.shape)
print(Y_pos.shape)

In [ ]:
H = load_data('/content/drive/My Drive/CTW_2019/h_Estimated_CTW_Train.h5')

print(H.shape)
H_Re = H[0,:,:,:]
H_Im = H[1,:,:,:]

data2 = np.sqrt(H_Re**2 + H_Im**2)
data2 = data2.reshape(17486,16,924)
print(data2.shape)
del H_Re, H_Im

In [ ]:
#Extracting polynomial features
#run only if needed

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
#%%
dim1 = int(data2.shape[0]/2)
#dim1 = 10
dim2 = data2.shape[1]
new_dim = 66
data3 = np.zeros((dim1,dim2, new_dim))

#Polynomial fitting code
t1 = time.time()
R2s = []
w = np.linspace(0,1,22)
for u in range(dim1):
    for v in range(dim2):
        
        degrees = [4,5,6,7,8]
        
        batches = [0,242,220,473,451,704,682,924]  #len = 8
        final_x = np.linspace(1,924,924)
        final_y = np.zeros((924,))
        prev = np.zeros([22,])
        for i in range(4):
            x = np.linspace(batches[i*2],batches[2*i + 1],-batches[i*2]+batches[2*i + 1])
           # print()
            #print(x.shape)
            y = []
        
            for j in range(batches[i*2],batches[2*i+1]):
                y.append(data2[u + dim1,v,j])
             
            y = np.array(y)
            x = x[:, np.newaxis]
            y = y[:, np.newaxis]
            r2 = np.Infinity
            best_deg = 0
            for deg in degrees:
                polynomial_features= PolynomialFeatures(degree=deg, interaction_only=False)
                x_poly = polynomial_features.fit_transform(x)
                model = LinearRegression()
                model.fit(x_poly, y)
                y_poly_pred = model.predict(x_poly)
                
                rmse = np.sqrt(mean_squared_error(y,y_poly_pred))
                #temp = r2_score(y,y_poly_pred)
                if rmse < r2:
                    best_deg = deg
                    r2 = rmse
                    final_y[batches[i*2]:batches[2*i+1]] = y_poly_pred.reshape((-1))
                    #print(str(r2) + " Testing ")
                    
            #print("Best found " + str(r2))
            R2s.append(rmse)
            
            tmp1 = final_y[batches[i*2]:batches[i*2]+22]
            if prev.all() != 0:
                final_y[batches[i*2]:batches[i*2]+22] = tmp1*w + prev*(1-w)
            prev = final_y[batches[i*2 + 1]-22:batches[i*2 + 1]]
            
        data3[u,v,:] = [final_y[i] for i in range(0,len(final_y), 14)]    
    t2 = time.time() 
    print(str(u) + "th of " + str(dim1))
    print("Time elasped: " + str((t2-t1)/60) + " mins.")    
    t2 = time.time() 
   # print(str(u) + "th of " + str(dim1))
   # print("Time elasped: " + str((t2-t1)/60) + " mins.")
#%%
t3 = time.time()
print("DONE")
print("Total time: " + str((t3-t1)/3600) + " hrs.")



In [ ]:
hf = h5py.File('/content/drive/My Drive/CTW_2019/data_feat2.h5', 'w')
hf.create_dataset('dataset_1', data=data3)
hf.close()

In [ ]:
#data_generator if disk dont have enough memory to run in large batches


from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.optimizers import Adadelta, Adam, RMSprop


def data_gen(data):
    for i in range(len(data)):
        yield (data[i:i+1],data[i:i+1])

def data_gen2(data, target):
    for i in range(len(data)):
        yield (data[i:i+1],target[i:i+1])
    

In [ ]:
#NN model

def mlp():
  inp = Input(shape=(16,66))
  x = Flatten()(inp)
  x = Dense(1024,activation = 'relu')(x)
  #x = Dense(1024, activation = 'relu')(x)
  x = Dropout(0.03)(x)
  x = Dense(512, activation = 'relu',kernel_initializer='glorot_uniform',
                bias_initializer='zeros')(x)
  x = Dropout(0.03)(x)
  x = Dense(256, activation = 'relu',kernel_initializer='glorot_uniform',
                bias_initializer='zeros')(x)
  x = Dropout(0.03)(x)
  x = Dense(128, activation = 'relu',kernel_initializer='glorot_uniform',
                bias_initializer='zeros')(x)
  x = Dropout(0.03)(x)
  x = Dense(32, activation = 'relu',kernel_initializer='glorot_uniform',
                bias_initializer='ones')(x)
  x = Dropout(0.03)(x)
  x = Dense(12, activation = 'relu',kernel_initializer='glorot_uniform',
                bias_initializer='ones')(x)
  x = Dropout(0.03)(x)
  op = Dense(3, activation = 'relu',kernel_initializer='glorot_uniform',
                bias_initializer='ones')(x)
  return Model(inp, op)



In [ ]:
#dividing the dataset for training and testing

from sklearn.model_selection import train_test_split
h_COMP = np.swapaxes(h_COMP,0,2)
Y_pos = np.swapaxes(Y_pos,0,1)
SNR = np.swapaxes(SNR,0,1)

X_train,X_test = train_test_split(h_COMP, test_size=0.1, random_state=42, shuffle=True)
y_train,y_test = train_test_split(Y_pos, test_size=0.1, random_state=42, shuffle=True)
SNR_train, SNR_test = train_test_split(SNR, test_size=0.1, random_state=42, shuffle=True)
#print(H_new.shape,pos.shape)

print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)
print(SNR_train.shape,SNR_test.shape)


In [ ]:
del h_COMP, SNR, Y_pos

In [ ]:
X_train, X_test = X_train[:,:,:,np.newaxis], X_test[:,:,:,np.newaxis]
print(X_train.shape,X_test.shape)


In [ ]:
#training

mlp_2019 = mlp()
from sklearn.model_selection import KFold
opt = RMSprop(learning_rate=0.001, decay=1e-6)
mlp_2019.compile(optimizer=opt,loss = 'mean_squared_error')
from keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience = 20)
checkpointer = ModelCheckpoint('/content/drive/My Drive/CTW_2019/CTW2019_mean_sqr2.h5', verbose=1, save_best_only=True)
mlp_2019.fit(X_train, y_train, epochs = 150, verbose=1, batch_size = 20, validation_split=0.1, callbacks = [checkpointer, early_stopping])



In [ ]:
def rmse(p1, p2):
  return np.sqrt(np.sum((p1-p2)**2, axis = 1))

In [ ]:
#checking error

m2  = load_model("/content/drive/My Drive/CTW_2019/CTW2019_mean_sqr2.h5")
l = 140
r = 150
pred = m2.predict(X_test)
org = y_test
print(pred[:,:-1].shape)
print(org[:,:-1].shape)
print(np.mean(rmse(pred, org)))
print(np.mean(rmse(pred[:,:-1], org[:,:-1])))

In [ ]:
error = np.array(error)
np.savetxt('/content/drive/My Drive/CTW_2019/test.out', x, delimiter=',')